In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
from Model import *
from sklearn.metrics import classification_report
import tensorflow_addons as tfa

In [2]:
keras.utils.set_random_seed(7)

In [3]:
train = pd.read_csv("../../dataset/goodreads_train.csv")
vocabulary2 = np.load('../../vocabulaires/voc_without_std_word_count_5.npy', allow_pickle=True)

In [ ]:
#model_list = [cnn1,cnn2, cnn3,cnn4,cnn5,cnn6, cnn8, cnn9, cnn10]
model_list = [transf5]
for model_obj in model_list:
    model = model_obj.Model(vocabulary2)
    chekpoint = keras.callbacks.ModelCheckpoint('checkpoint/trans5', save_weights_only=True,
    monitor='val_f1_score',
    mode='max',
    save_best_only=True)
    def scheduler(epoch, lr):
        if epoch < 4:
            return lr
        else:
            return lr * tf.math.exp(-0.1)
    model.model.compile(optimizer=keras.optimizers.Adamax(learning_rate=0.001),
                       loss=keras.losses.categorical_crossentropy,
                       metrics=[keras.metrics.categorical_accuracy, tfa.metrics.F1Score(num_classes=6, average='weighted')]
                       )
    print(model.name)
    print(model.model.summary())
    if not os.path.exists(f"logs/{model.name}"):
        os.mkdir(f"logs/{model.name}")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"logs/{model.name}")
    model.run_experiment([train['review_text'], train['n_comments'], train['n_votes']], train['rating'], epochs=12, callbacks=[keras.callbacks.LearningRateScheduler(scheduler,0),tensorboard_callback,chekpoint], batch_size=30)

transf4
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization (TextVector  (None, 192)         0           ['input_1[0][0]']                
 ization)                                                                                         
                                                                                                  
 token_and_position_embedding (  (None, 192, 300)    32907300    ['text_vectorization[0][0]']     
 TokenAndPositionEmbedding)                                                                       
                                                                                      

In [ ]:
model.model.save('transf5')